In [12]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import linecache
from google.cloud import storage

# RAW EXAMPLE

Example taken from the simplfied database on : 
https://console.cloud.google.com/storage/browserquickdraw_dataset/full/raw

In [ ]:
#RAW DATA: Example 
df = pd.DataFrame()
with open(data_path, 'r') as f:
     for i, line in enumerate(f):
            if i > 0:
                break

            json_line = json.loads(line)
            df = pd.concat([df, pd.DataFrame(json_line)], ignore_index=True)
#             print(type(json_line))
#             print(json_line.keys())
#             print(json_line['word'], len(json_line['drawing']))
            coords = json_line['drawing']
    
            for vector in coords:
                print(len(vector))
                print(vector)
           

In [ ]:
df

In [ ]:
df["drawing"][0][0:1]

In [ ]:
drawing_test= df["drawing"][0][2]
drawing_test

In [ ]:

# Extract data from the first row
first_row = df.loc[0]  # Change the index if needed

# Extract x, y, and time lists
x_values = first_row['drawing'][0]
y_values = first_row['drawing'][1]

# Create a scatter plot of the points
plt.scatter(x_values, y_values, color='blue', marker='o', label='Points')

# Connect the points with lines
for i in range(len(x_values) - 1):
    plt.plot([x_values[i], x_values[i + 1]], [y_values[i], y_values[i + 1]], color='red')

# Customize the plot (add labels, title, etc.)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('Drawing from First RAW Row')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Extract data from the first drawing 



# Create a new figure for the combined drawing
plt.figure(figsize=(8, 6))

# Iterate through each row and plot the points
num_rows = df.shape[0]
for i in range(num_rows):
    x_values = df.loc[i, 'drawing'][0]
    y_values = df.loc[i, 'drawing'][1]
    plt.plot(x_values, y_values, label=f"Row {i}")

    # Create a scatter plot of the points
    plt.scatter(x_values, y_values, color='blue', marker='o', label='Points')

# Connect the points with lines
for i in range(len(x_values) - 1):
    plt.plot([x_values[i], x_values[i + 1]], [y_values[i], y_values[i + 1]], color='red')

# Customize the plot (add labels, title, etc.)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('Drawing from First RAW drawing')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

# SIMPLIFIED EXAMPLE

Example taken from the simplfied database on :
https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/simplified

In [ ]:
#command to download the examle in terminal: gsutil -m cp 'gs://quickdraw_dataset/full/simplified/axe.ndjson' .

In [ ]:
# # Download this face file (simplified) and rename it full_simplified_face:
# gsutil -m cp 'gs://quickdraw_dataset/full/simplified/face.ndjson' full_simplified_face.ndjson


In [13]:
path_data = '/Users/loicregne/code/rs-uk/raw_data'
file_name = 'full_simplified_face.ndjson'

# Processing Simplified file

In [14]:
##Define the function to process the Simplified file and return an array and class name (the drawing to guess)
def parse_line(json_drawing: json) -> dict:
    """Parse an ndjson line and return ink (as np array) and classname."""
    class_name = json_drawing["word"]
    inkarray = json_drawing["drawing"]
    stroke_lengths = [len(stroke[0]) for stroke in inkarray]
    total_points = sum(stroke_lengths)
    np_ink = np.zeros((total_points, 3), dtype=np.float32)
    current_t = 0
    for stroke in inkarray:
        for i in [0, 1]:
            np_ink[current_t:(current_t + len(stroke[0])), i] = stroke[i]
        current_t += len(stroke[0])
        np_ink[current_t - 1, 2] = 1  # stroke_end

    # Preprocessing.
    # 1. Size normalization.
    lower = np.min(np_ink[:, 0:2], axis=0)
    upper = np.max(np_ink[:, 0:2], axis=0)
    scale = upper - lower
    scale[scale == 0] = 1
    np_ink[:, 0:2] = (np_ink[:, 0:2] - lower) / scale

    # 2. Compute deltas.
    np_ink[1:, 0:2] -= np_ink[0:-1, 0:2]
    np_ink = np_ink[1:, :]
    return np.round(np_ink,5), class_name

In [37]:
##Define the function to process the Simplified file and return an array and class name (the drawing to guess)
def get_drawing_data(json_drawing: json) -> np.array:
    '''
    Extracts the drawing data (strokes list) from a drawing JSON file.
    Transforms the strokes from coordinates to deltas.
    Returns an np.array of deltas (d_x, d_y, end_of_stroke)
    '''
    # --- Data extraction ---
    lst_strokes = json_drawing['drawing']
    
    x = []
    y = []
    stroke_delimiter = []
    lst_points = [x, y, stroke_delimiter]
    
    for stroke in lst_strokes:
        # Creating the third list to pass to the model with 0 all along and a 1 at the end of the stroke
        stroke_delimiter = [0] * len(stroke[0])
        stroke_delimiter[-1] = 1
        # Appending x, y, and the delimiter to the new list of points
        lst_points[0] += stroke[0]
        lst_points[1] += stroke[1]
        lst_points[2] += stroke_delimiter

    np_points = np.asarray(lst_points)
    np_points = np_points.T

    # --- Preprocessing ---
    # 1. Size normalization
    lower = np.min(np_points[:, 0:2], axis=0) # returns (x_min, y_min)
    upper = np.max(np_points[:, 0:2], axis=0) # returns (x_max, y_max)
    scale = upper - lower # returns (width, heigth)
    scale[scale == 0] = 1
    np_points[:, 0:2] = (np_points[:, 0:2] - lower) / scale

    # 2. Compute deltas
    np_points[1:, 0:2] -= np_points[0:-1, 0:2]
    np_deltas = np_points[1:, :]
    return np.round(np_deltas,4)

# Define the function to process a  CLASS (one of the word to guess) full file 

In [15]:
ndjson_path = path_data + '/' + file_name

def drawing_extraction(ndjson_path, nb_drawings_to_load):
    dict_drawings = {}  # Initialize an empty dictionary
    with open(ndjson_path, 'r') as f:
        for i, line in enumerate(f):
            json_line = json.loads(line)
            np_ink, class_name = parse_line(json_line)
            #np_ink= np.round(np_ink,5)
            #print(np_ink)
            length_np_ink = len(np_ink)
            dict_drawing_info = {
                "class": class_name,
                "drawing_nb": i,
                "length_np_ink": length_np_ink,
                "np_ink": np_ink.tolist()  #need to be transformed to list to dump as Json file later

            }
            #print(image_info["np_ink"])
            dict_drawings[json_line["key_id"]] = dict_drawing_info
            i += 1
            if i == nb_drawings_to_load:
                break
    return dict_drawings

In [54]:
# Trying a more efficient way to load the data

def drawings_extraction_fast(ndjson_path: object, nb_drawings_to_load: str) -> list:
    '''
    Extract drawing(s) information from a list of JSON drawings (as NDJSON),
    as a list of dictionaries. We specify the number of drawings to load (in
    order of the NDJSON) as a string (number or 'all'). Each dictionary contains:
        - key_id, as integer
        - class, as a string
        - length, as integer
        - lst_delats, as list
    '''
    lst_drawings = []  # Initialize the list to return

    if nb_drawings_to_load.isnumeric():
        for i in range(0, int(nb_drawings_to_load)):
            json_drawing = json.loads(linecache.getline(ndjson_path, i+1 , module_globals=None))
            np_deltas = get_drawing_data(json_drawing)
            dict_drawing = {'key_id': json_drawing['key_id'],
                            'class': json_drawing['word'],
                            'length': len(np_deltas),
                            'lst_deltas': np_deltas.tolist()  # need to be transformed to list to dump as Json file later
                           }
            lst_drawings.append(dict_drawing)
    elif nb_drawings_to_load == 'all':
        i = 0
        # TODO: change the method to find last line of ndjson file!
        while len(linecache.getline(ndjson_path, i+1 , module_globals=None))>0:
            json_drawing = json.loads(linecache.getline(ndjson_path, i+1 , module_globals=None))
            linecache.clearcache()
            np_deltas = get_drawing_data(json_drawing)
            dict_drawing = {'key_id': json_drawing['key_id'],
                            'class': json_drawing['word'],
                            'length': len(np_deltas),
                            'lst_deltas': np_deltas.tolist()  # need to be transformed to list to dump as Json file later
                           }
            lst_drawings.append(dict_drawing)
            i += 1
    else:
        return
        
    return lst_drawings

In [29]:
linecache.getline(ndjson_path, 162000, module_globals=None)

''

In [21]:
ndjson_path

'/Users/loicregne/code/rs-uk/raw_data/full_simplified_face.ndjson'

In [49]:
%%time
dict_test = drawing_extraction(ndjson_path, 100000)

CPU times: user 5.46 s, sys: 1.24 s, total: 6.7 s
Wall time: 7.25 s


In [57]:
%%time
dict_test = drawings_extraction_fast(ndjson_path, '100000')

CPU times: user 4.57 s, sys: 114 ms, total: 4.69 s
Wall time: 4.7 s


In [13]:
len(dict_test)

50000

In [14]:
# Where we want to save the output file and a name is given
output_file = path_data + '/' + 'processed_full_simplified_face.json'

In [15]:
# Function to save the dictionary file for the processed images
def save_images_dict_to_json(images_dict, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(images_dict, json_file)

In [16]:
save_images_dict_to_json(dict_test,output_file)

In [17]:
len(dict_test)

50000

In [5]:
def upload_blob(bucket_name, file_path, file_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_path + '/' + file_name)

In [8]:
bucket_name = 'quickdraw-simplified-processed'
file_path = path_data
file_name = 'processed_full_simplified_face.json'

upload_blob(bucket_name, file_path, file_name)